## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility

### Setup Common Resources

In [2]:
region = 'ca-west-1'
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

In [3]:
setup_dna = False
if setup_dna:
    # Boto3 Direct Calls
    direct_calls_dir = "./dna_visualization/external_database/boto3_direct" # Boto3 only direct calls
    deployment_utility.deploy_experiment(direct_calls_dir)

    # Boto3 SNS calls
    sns_calls_dir = "./dna_visualization/external_database/boto3_sns" # Boto3 only sns calls
    deployment_utility.deploy_experiment(sns_calls_dir)

    # AWS Step Function
    aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
    deployment_utility.deploy_experiment(aws_step_functions_dir)

    # Multi-X
    multi_x_dir = "./dna_visualization/external_database/multi_x" # Multi_x
    deployment_utility.deploy_experiment(multi_x_dir)

### Image Processing

In [4]:
setup_image_processing = False

if setup_image_processing:
    # Boto3 Direct Calls
    direct_calls_dir = "./image_processing/external_database/boto3_direct" # Boto3 only direct calls
    deployment_utility.deploy_experiment(direct_calls_dir)

    # Boto3 SNS calls
    sns_calls_dir = "./image_processing/external_database/boto3_sns" # Boto3 only sns calls
    deployment_utility.deploy_experiment(sns_calls_dir)

    # AWS Step Function
    aws_step_functions_dir = "./image_processing/external_database/aws_step_function" # AWS Step Functions
    deployment_utility.deploy_experiment(aws_step_functions_dir)

    # Multi-X
    multi_x_dir = "./image_processing/external_database/multi_x" # Multi_x
    deployment_utility.deploy_experiment(multi_x_dir)

### Video Analytics

In [6]:
setup_video_analysis = True

if setup_video_analysis:
    # # Multi-X
    # multi_x_dir = "./video_analytics/external_database/multi_x" # Multi_x
    # deployment_utility.deploy_experiment(multi_x_dir)

    # Boto3 Direct Calls
    direct_calls_dir = "./video_analytics/external_database/boto3_direct" # Boto3 only direct calls
    deployment_utility.deploy_experiment(direct_calls_dir)


Deploying boto3_direct workload
wo-vid_an-ed-direct

Deploying wo-vid_an-ed-direct-streaming
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-vid_an-ed-direct-streaming-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 108.55kB done
#4 DONE 0.0s

#5 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:073bef913e4144bba15fabed2ab4a42d2881d1cee7e5c9da0ba78be0ddcfbd4b
#5 CACHED

#6 [2/6] COPY requirements.txt ./
#6 DONE 0.0s

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 0.693 Collecting opencv-python-headless==4.5.5.64
#7 60.40   Downloading opencv_python_headless-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.8 MB)
#7 60.93      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 98.5 MB/s eta 0:00:00
#7 61.03 Collecting torchvision==0.17.1
#7 61.04   Downloading torchvision-0.17.1-cp38-cp3

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-direct-streaming]
885ed6d5b398: Preparing
1305f66fd568: Preparing
d4c0a55a1dfd: Preparing
821cf7d50d17: Preparing
83463f895008: Preparing
3a5edae477ec: Preparing
2d18318c7f6d: Preparing
59e530c8b37a: Preparing
6c8d7ede4ae8: Preparing
3a5edae477ec: Waiting
2d18318c7f6d: Waiting
59e530c8b37a: Waiting
b30d0951e27f: Preparing
25a97811023e: Preparing
6c8d7ede4ae8: Waiting
25a97811023e: Waiting
b30d0951e27f: Waiting
1305f66fd568: Pushed
d4c0a55a1dfd: Pushed
885ed6d5b398: Pushed
83463f895008: Pushed
6c8d7ede4ae8: Pushed
59e530c8b37a: Pushed
b30d0951e27f: Pushed
3a5edae477ec: Pushed
2d18318c7f6d: Pushed
25a97811023e: Pushed
821cf7d50d17: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-direct-streaming:latest to ECR.


latest: digest: sha256:3800561451cfcb23f1962ccf48600fdd9bc2424b1fbca3cb74e3fb839ddf2b6e size: 2626
Resulting Function ARN: arn:aws:lambda:ca-west-1:485595969306:function:wo-vid_an-ed-direct-streaming

Deploying wo-vid_an-ed-direct-decode
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-vid_an-ed-direct-decode-policy
Creating lambda function


...

latest: digest: sha256:76f7196bbf65fb4dd658a964f4df27860fec5c4c3e540f9f18a78a43febbc3cc size: 2626
Resulting Function ARN: arn:aws:lambda:ca-west-1:485595969306:function:wo-vid_an-ed-direct-recognition

Deploying wo-vid_an-ed-direct-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-vid_an-ed-direct-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.3s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:073bef913e4144bba15fabed2ab4a42d2881d1cee7e5c9da0ba78be0ddcfbd4b
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 107.81kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:e1e02404bc3acf8ede32c5b55636295837385b8e242eff57b01e5488406e0e12 done
#11 naming to docker.io/library/wo-vid_an-ed-dire

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-direct-get_input]
b4cbb7852248: Preparing
b4fe1d1a4599: Preparing
234381dfc1df: Preparing
821cf7d50d17: Preparing
83463f895008: Preparing
3a5edae477ec: Preparing
2d18318c7f6d: Preparing
3a5edae477ec: Waiting
2d18318c7f6d: Waiting
59e530c8b37a: Preparing
6c8d7ede4ae8: Preparing
b30d0951e27f: Preparing
25a97811023e: Preparing
59e530c8b37a: Waiting
b30d0951e27f: Waiting
6c8d7ede4ae8: Waiting
b4cbb7852248: Pushed
b4fe1d1a4599: Pushed
234381dfc1df: Pushed
83463f895008: Pushed
6c8d7ede4ae8: Pushed
59e530c8b37a: Pushed
b30d0951e27f: Pushed
3a5edae477ec: Pushed
2d18318c7f6d: Pushed
25a97811023e: Pushed
821cf7d50d17: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-direct-get_input:latest to ECR.


latest: digest: sha256:050790e8d4006f31d0cd1fcb3d8e2f82e649ea6721e7329d5fedc9f8512e3976 size: 2625
Resulting Function ARN: arn:aws:lambda:ca-west-1:485595969306:function:wo-vid_an-ed-direct-get_input
Completed deployment of wo-vid_an-ed-direct




In [7]:
multi_x_dir = "./video_analytics/external_database/multi_x" # Multi_x
deployment_utility.deploy_experiment(multi_x_dir)

Found credentials in environment variables.


Deploying multi_x workload
wo-vid_an-ed-multi_x-0.0.2


Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.


Removed workflow wo-vid_an-ed-multi_x-0.0.2


Deploying workflow wo-vid_an-ed-multi_x with version 0.0.2
Found credentials in environment variables.
Building deployment package
Building deployment plan
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Executing deployment plan
Found credentials in environment variables.
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 326B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.11
#2 DONE 0.3s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 116.39kB done
#4 DONE 0.0s

#5 [1/6] FROM public.ecr.aws/lambda/python:3.11@sha256:8e0ee76e44c44fcb23359a9abf8cc471a01fbf1579677b66e3479f2b635c13ac
#5 CACHED

#6 [2/6] COPY requirements.txt ./
#6 DONE 0.0s

#7 [3/6] RUN

...

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-multi_x-0_0_2-recognition_aws-ca-west-1]
0b9f323ab1c4: Preparing
c62e451a6466: Preparing
0110844346aa: Preparing
dc8b3dc8c43a: Preparing
1f47c8c59cfd: Preparing
8cc4aea68aa7: Preparing
2c337431d67d: Preparing
59e530c8b37a: Preparing
7d36fc7e830b: Preparing
e31dd85b1b4e: Preparing
25a97811023e: Preparing
2c337431d67d: Waiting
59e530c8b37a: Waiting
7d36fc7e830b: Waiting
e31dd85b1b4e: Waiting
25a97811023e: Waiting
8cc4aea68aa7: Waiting
c62e451a6466: Pushed
1f47c8c59cfd: Pushed
0110844346aa: Pushed
0b9f323ab1c4: Pushed
8cc4aea68aa7: Pushed
7d36fc7e830b: Pushed
59e530c8b37a: Pushed
e31dd85b1b4e: Pushed
2c337431d67d: Pushed
25a97811023e: Pushed
dc8b3dc8c43a: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.ca-west-1.amazonaws.com/wo-vid_an-ed-multi_x-0_0_2-recognition_aws-ca-west-1:latest to ECR.


latest: digest: sha256:8af577b8f7f85137cd80a88345bf4de145f9ab71f14752e0c8081e1e88de632a size: 2624


Uploading workflow to configuration server
Found credentials in environment variables.
Workflow wo-vid_an-ed-multi_x with version 0.0.2 deployed
Workflow id: wo-vid_an-ed-multi_x-0.0.2


Completed deployment of wo-vid_an-ed-multi_x-0.0.2


